# LaTeX table of dynamic rupture model parameters

Generate the dynamic rupture model parameters.

2025.3.3 Kurama Okubo

- 2025.3.26 update for master table

In [1]:
import os
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
%matplotlib inline
import numpy as np
import pandas as pd
from datetime import timedelta
from tqdm import tqdm
import warnings
import time

import scipy.integrate as integrate
import seaborn as sns 

%load_ext autoreload
%autoreload 2

plt.rcParams["font.family"] = 'Arial'
# plt.rcParams["font.sans-serif"] = "DejaVu Sans, Arial, Helvetica, Lucida Grande, Verdana, Geneva, Lucid, Avant Garde, sans-serif"
plt.rcParams["font.size"] = 11
plt.rcParams["xtick.direction"] = "in"
plt.rcParams["xtick.major.size"] = 4.75
plt.rcParams["xtick.major.width"] = 0.75
plt.rcParams["xtick.minor.size"] = 3
plt.rcParams["xtick.minor.width"] = 0.4
plt.rcParams["xtick.minor.visible"] = True

plt.rcParams["ytick.direction"] = "in"
plt.rcParams["ytick.major.size"] = 4.75
plt.rcParams["ytick.major.width"] = 0.75
plt.rcParams["ytick.minor.size"] = 3
plt.rcParams["ytick.minor.width"] = 0.4
plt.rcParams["ytick.minor.visible"] = True

plt.rcParams["savefig.transparent"] = True

plt.rcParams['axes.linewidth'] = 0.75

# Read the datasheet

In [2]:
a_patch = 4.0e-3 # patch radius without margine
rupturetype = "pulse" # "crack": without self-healing or "pulse": with self-healing
p_dcscaleexp = 0.6 #0.55 #0.54 #0.56 #0.555  # #0.475 #0.55 #0.575 #0.65 #0.8 #0.65 #0.7

IfInitparam = True # True to make the input files with the short time duration to output the initial condition

casestudy_name = f"a={a_patch*1e3:.2f}_ruptype={rupturetype:s}_pdcscaling={p_dcscaleexp:.3f}"
print(casestudy_name)

a=4.00_ruptype=pulse_pdcscaling=0.600


In [3]:
hat_sn_patch = 6e6
hat_fr_patch =0.3
stressbackground_beta = 0.35


casename = casestudy_name+"_sn={:.1f}MPa_hatfr={:.1f}_bgbeta={:.2f}".format(hat_sn_patch/1e6, hat_fr_patch, stressbackground_beta)


In [4]:
df_dynparam = pd.read_csv(f"../data/gouge_dynamicrupture_modelparam_{casename}_master.csv", index_col=0)

In [5]:
target_events = [24, 50, 52, 72 ,129]

In [6]:
df_dynparam_targets = df_dynparam.loc[df_dynparam.index.isin(target_events), :].copy()
df_dynparam_targets.index = [f"M{x}" for x in df_dynparam_targets.index]

In [7]:
df_dynparam_targets

,M0_mean,Tw_mean,Mw_mean,hat_sn_patch,hat_sn_background,delsig_withmargin_try,slip_try,initialstress_fraction,delsigma_factor,fp_patch,...,GIIC_patch,GIIC_background,Sratio_nuc,Sratio_patch,Ainit_nuc,Ainit_patch,Anuc_nuc,Anuc_patch,rnuc_nuc,rnuc_patch
M24,0.065639,0.000002,-6.858558,6000000.0,2000000.0,422824.778,3.258138e-08,0.980,0.695,0.356099,...,0.002173,0.0,0,0.145414,3.82,3.827762,0.000008,0.000008,1.629693,1.631348
M50,0.326376,0.000003,-6.394187,6000000.0,2000000.0,2102405.090,1.620039e-07,0.925,0.500,0.513730,...,0.021675,0.0,0,0.219918,3.82,3.844819,0.000004,0.000004,1.119773,1.123405
M52,0.481138,0.000003,-6.281820,6000000.0,2000000.0,3099325.440,2.388230e-07,0.925,0.460,0.581205,...,0.035995,0.0,0,0.183450,3.82,3.834911,0.000004,0.000004,1.074244,1.076338
M72,0.808650,0.000003,-6.131493,6000000.0,2000000.0,5209047.070,4.013907e-07,0.925,0.430,0.727908,...,0.074794,0.0,0,0.146239,3.82,3.827886,0.000003,0.000003,0.963985,0.964980
M129,1.131238,0.000003,-6.034297,6000000.0,2000000.0,7287045.040,5.615139e-07,0.925,0.410,0.862647,...,0.120289,0.0,0,0.129930,3.82,3.825657,0.000003,0.000003,0.896729,0.897393


# Compute additional values

In [8]:
# Compute actual stress drop
df_dynparam_targets.loc[:, "$\\Delta \\tau$"] = df_dynparam_targets.apply(lambda x: x.delsig_withmargin_try * x.delsigma_factor, axis=1)

# Ds
hat_ds_factor_rapidnuc_patch = 5.5

df_dynparam_targets.loc[:, "$D_s$"] = df_dynparam_targets.apply(lambda x: x.dc_try * hat_ds_factor_rapidnuc_patch, axis=1)


# Reformat table for latex output

In [9]:
df_dynparam_targets_selected = df_dynparam_targets[["M0_mean", "Tw_mean", "Mw_mean", "hat_sn_patch", "delsig_withmargin_try", "slip_try", "initialstress_fraction", 
                                                    "delsigma_factor", "tau_0_patch", "$\\Delta \\tau$", "fp_patch", "hat_fr", "dc_try", "$D_s$",
                                                    "GIIC_patch", "rnuc_patch"]].copy()

In [10]:
# df_dynparam_targets_selected.index.name = "GP event ID"

## Compute the additional values

In [11]:
df_dynparam_targets_selected

,M0_mean,Tw_mean,Mw_mean,hat_sn_patch,delsig_withmargin_try,slip_try,initialstress_fraction,delsigma_factor,tau_0_patch,$\Delta \tau$,fp_patch,hat_fr,dc_try,$D_s$,GIIC_patch,rnuc_patch
M24,0.065639,0.000002,-6.858558,6000000.0,422824.778,3.258138e-08,0.980,0.695,2093863.22,2.938632e+05,0.356099,0.3,1.291319e-08,7.102256e-08,0.002173,1.631348
M50,0.326376,0.000003,-6.394187,6000000.0,2102405.090,1.620039e-07,0.925,0.500,2851202.54,1.051203e+06,0.513730,0.3,3.380393e-08,1.859216e-07,0.021675,1.123405
M52,0.481138,0.000003,-6.281820,6000000.0,3099325.440,2.388230e-07,0.925,0.460,3225689.70,1.425690e+06,0.581205,0.3,4.266755e-08,2.346715e-07,0.035995,1.076338
M72,0.808650,0.000003,-6.131493,6000000.0,5209047.070,4.013907e-07,0.925,0.430,4039890.24,2.239890e+06,0.727908,0.3,5.826293e-08,3.204461e-07,0.074794,0.964980
M129,1.131238,0.000003,-6.034297,6000000.0,7287045.040,5.615139e-07,0.925,0.410,4787688.46,2.987688e+06,0.862647,0.3,7.126368e-08,3.919502e-07,0.120289,0.897393


## Rename the columns

In [16]:
df_dynparam_targets_selected.rename(
    columns={"M0_mean":"$M_0^{\\text{obs}}$", 
             "Tw_mean":"$T_w^{\\text{obs}}$",
             "Mw_mean":"$M_w^{\\text{obs}}$",
             "hat_sn_patch":"$\\sigma_n^{\\text{PHC}}$",
             "delsig_withmargin_try": "$\\Delta \\sigma_{\\text{ref}}$",
             "slip_try": "$\\bar{u}_0$",
             "initialstress_fraction": "$c$",
             "delsigma_factor": "$s$",
             "fp_patch": "$\\mu_s$",
             "hat_fr": "$\\mu_d$",
             "dc_try": "$D_c$",
             "tau_0_patch": "$\\tau_0^{\\text{PHC}}$",
             "GIIC_patch": "$G_{IIC}$",
             "rnuc_patch": "$R_{c}$",
            },
    inplace=True)

In [17]:
df_dynparam_targets_selected

,$M_0^{\text{obs}}$,$T_w^{\text{obs}}$,$M_w^{\text{obs}}$,$\sigma_n^{\text{PHC}}$,$\Delta \sigma_{\text{ref}}$,$\bar{u}_0$,$c$,$s$,$\tau_0^{\text{PHC}}$,$\Delta \tau$,$\mu_s$,$\mu_d$,$D_c$,$D_s$,$G_{IIC}$,$R_{c}$
M24,0.065639,0.000002,-6.858558,6000000.0,422824.778,3.258138e-08,0.980,0.695,2093863.22,2.938632e+05,0.356099,0.3,1.291319e-08,7.102256e-08,0.002173,1.631348
M50,0.326376,0.000003,-6.394187,6000000.0,2102405.090,1.620039e-07,0.925,0.500,2851202.54,1.051203e+06,0.513730,0.3,3.380393e-08,1.859216e-07,0.021675,1.123405
M52,0.481138,0.000003,-6.281820,6000000.0,3099325.440,2.388230e-07,0.925,0.460,3225689.70,1.425690e+06,0.581205,0.3,4.266755e-08,2.346715e-07,0.035995,1.076338
M72,0.808650,0.000003,-6.131493,6000000.0,5209047.070,4.013907e-07,0.925,0.430,4039890.24,2.239890e+06,0.727908,0.3,5.826293e-08,3.204461e-07,0.074794,0.964980
M129,1.131238,0.000003,-6.034297,6000000.0,7287045.040,5.615139e-07,0.925,0.410,4787688.46,2.987688e+06,0.862647,0.3,7.126368e-08,3.919502e-07,0.120289,0.897393


# Dump the output

## unit Convert 

In [18]:
df_dynparam_targets_selected.columns

Index(['$M_0^{\text{obs}}$', '$T_w^{\text{obs}}$', '$M_w^{\text{obs}}$',
       '$\sigma_n^{\text{PHC}}$', '$\Delta \sigma_{\text{ref}}$',
       '$\bar{u}_0$', '$c$', '$s$', '$\tau_0^{\text{PHC}}$', '$\Delta \tau$',
       '$\mu_s$', '$\mu_d$', '$D_c$', '$D_s$', '$G_{IIC}$', '$R_{c}$'],
      dtype='object')

In [21]:
df_dynparam_targets_selected["$T_w^{\\text{obs}}$"]


M24     0.000002
M50     0.000003
M52     0.000003
M72     0.000003
M129    0.000003
Name: $T_w^{\text{obs}}$, dtype: float64

In [22]:
s2mus = lambda x: x*1e6
Pa2MPa = lambda x: x/1e6
m2um = lambda x: x*1e6


In [23]:
# Show GIIC with the power of 10
def GIIC_latex_conv(x):
    # ref: https://stackoverflow.com/a/59744605
    sci_notation = "{:.2e}".format(x)
    value, expscale = sci_notation.split("e")
    return "${0} \\times 10^{{{1}}}$".format(float(value), int(expscale))


In [24]:
x = 0.021406
GIIC_latex_conv(x)

'$2.14 \\times 10^{-2}$'

In [29]:
# Unit conversion
formatters_targets = {"$M_0^{\\text{obs}}$": lambda x: '%.2f' % x,
                      "$T_w^{\\text{obs}}$": lambda x: '%.2f' % s2mus(x),
                      "$M_w^{\\text{obs}}$": lambda x: '%.1f' % x,
                      "$\\sigma_n^{\\text{PHC}}$": lambda x: '%.1f' % Pa2MPa(x),
                      "$\\Delta \\sigma_{\\text{ref}}$": lambda x: '%.2f' % Pa2MPa(x),
                      "$\\bar{u}_0$": lambda x: '%.2f' % m2um(x),
                      "$c$": lambda x: '%.3f' % x,
                      "$s$": lambda x: '%.3f' % x,
                      "$\\tau_0^{\\text{PHC}}$":  lambda x: '%.2f' % Pa2MPa(x),
                      "$\\Delta \\tau$":  lambda x: '%.2f' % Pa2MPa(x),
                      "$\\mu_s$": lambda x: '%.2f' % x,
                      "$\\mu_d$": lambda x: '%.1f' % x,
                      "$D_c$": lambda x: '%.3f' % m2um(x),
                      "$D_s$": lambda x: '%.3f' % m2um(x),
                      "$G_{IIC}$": lambda x: '%.2f $\\times$ 10$^{{%d}}$' % (float(f"{x:.2e}".split("e")[0]), float(f"{x:.2e}".split("e")[1])) ,
                      "$R_{c}$": lambda x: '%.2f' % x,
                     }

In [30]:
latextable_targets = df_dynparam_targets_selected.to_latex(header=True, column_format=f"l"+"c"*(len(df_dynparam_targets_selected.columns)), formatters= formatters_targets)


In [31]:
print(latextable_targets)

\begin{tabular}{lcccccccccccccccc}
\toprule
 & $M_0^{\text{obs}}$ & $T_w^{\text{obs}}$ & $M_w^{\text{obs}}$ & $\sigma_n^{\text{PHC}}$ & $\Delta \sigma_{\text{ref}}$ & $\bar{u}_0$ & $c$ & $s$ & $\tau_0^{\text{PHC}}$ & $\Delta \tau$ & $\mu_s$ & $\mu_d$ & $D_c$ & $D_s$ & $G_{IIC}$ & $R_{c}$ \\
\midrule
M24 & 0.07 & 2.46 & -6.9 & 6.0 & 0.42 & 0.03 & 0.980 & 0.695 & 2.09 & 0.29 & 0.36 & 0.3 & 0.013 & 0.071 & 2.17 $\times$ 10$^{{-3}}$ & 1.63 \\
M50 & 0.33 & 2.58 & -6.4 & 6.0 & 2.10 & 0.16 & 0.925 & 0.500 & 2.85 & 1.05 & 0.51 & 0.3 & 0.034 & 0.186 & 2.17 $\times$ 10$^{{-2}}$ & 1.12 \\
M52 & 0.48 & 2.60 & -6.3 & 6.0 & 3.10 & 0.24 & 0.925 & 0.460 & 3.23 & 1.43 & 0.58 & 0.3 & 0.043 & 0.235 & 3.60 $\times$ 10$^{{-2}}$ & 1.08 \\
M72 & 0.81 & 2.61 & -6.1 & 6.0 & 5.21 & 0.40 & 0.925 & 0.430 & 4.04 & 2.24 & 0.73 & 0.3 & 0.058 & 0.320 & 7.48 $\times$ 10$^{{-2}}$ & 0.96 \\
M129 & 1.13 & 2.63 & -6.0 & 6.0 & 7.29 & 0.56 & 0.925 & 0.410 & 4.79 & 2.99 & 0.86 & 0.3 & 0.071 & 0.392 & 1.20 $\times$ 10$^{{-1}}

In [32]:
df_dynparam_targets_selected

,$M_0^{\text{obs}}$,$T_w^{\text{obs}}$,$M_w^{\text{obs}}$,$\sigma_n^{\text{PHC}}$,$\Delta \sigma_{\text{ref}}$,$\bar{u}_0$,$c$,$s$,$\tau_0^{\text{PHC}}$,$\Delta \tau$,$\mu_s$,$\mu_d$,$D_c$,$D_s$,$G_{IIC}$,$R_{c}$
M24,0.065639,0.000002,-6.858558,6000000.0,422824.778,3.258138e-08,0.980,0.695,2093863.22,2.938632e+05,0.356099,0.3,1.291319e-08,7.102256e-08,0.002173,1.631348
M50,0.326376,0.000003,-6.394187,6000000.0,2102405.090,1.620039e-07,0.925,0.500,2851202.54,1.051203e+06,0.513730,0.3,3.380393e-08,1.859216e-07,0.021675,1.123405
M52,0.481138,0.000003,-6.281820,6000000.0,3099325.440,2.388230e-07,0.925,0.460,3225689.70,1.425690e+06,0.581205,0.3,4.266755e-08,2.346715e-07,0.035995,1.076338
M72,0.808650,0.000003,-6.131493,6000000.0,5209047.070,4.013907e-07,0.925,0.430,4039890.24,2.239890e+06,0.727908,0.3,5.826293e-08,3.204461e-07,0.074794,0.964980
M129,1.131238,0.000003,-6.034297,6000000.0,7287045.040,5.615139e-07,0.925,0.410,4787688.46,2.987688e+06,0.862647,0.3,7.126368e-08,3.919502e-07,0.120289,0.897393
